In [1]:
from flask import Flask, render_template, request
import pandas as pd
import os
import shutil
import torch
from IPython.display import Image, clear_output  # to display images
import cv2
import tqdm
import numpy as np
import glob
import sys
sys.path.insert(0, './yolov5')
from utils.datasets import *
from utils.utils import *
from fastai import *
from fastai.vision import *
from script import predict_plates,get_string_from_image
import sqlite3
from sqlite3 import Error

Loading model successfully...
[INFO] Model loaded successfully...
[INFO] Labels loaded successfully...


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def detect():
    os.chdir('C:\\Users\\Muhammad Ahmed\\Desktop\\DS\\Hackathon\\')
    vidcap = cv2.VideoCapture('yolov5/video/HD CCTV Traffic.mp4')
    success,image = vidcap.read()
    count = 0
    while success:
        cv2.imwrite("yolov5/converted/frame%d.jpg" % count, image)     # save frame as JPEG file      
        success,image = vidcap.read()
        count += 1

    os.chdir("yolov5")
    os.system('python detect.py --weights last_yolov5s_results.pt --img 416 --conf 0.4 --source converted')

    list_frames = pd.DataFrame(columns=['Path','Number'])
    
    list_frames['Path'] = sorted(glob.glob('inference/output/*.jpg'))

    list_frames['Number'] = list_frames['Path'].apply(lambda x: x.split('frame')[1].split('.')[0]).astype('int')

    list_frames = list_frames.sort_values('Number').reset_index(drop = True)

    img_array = []

    for filename in list_frames['Path']:
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)


    out = cv2.VideoWriter('Extracted/project.mp4',cv2.VideoWriter_fourcc(*'H264'), 25,size)

    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

    owd = os.getcwd()
    os.chdir('C:\\Users\\Muhammad Ahmed\\Desktop\\DS\\Hackathon\\')

    shutil.copy('yolov5/Extracted/project.mp4', 'static/Extracted/project.mp4')

In [4]:
def extract():
    os.chdir('C:\\Users\\Muhammad Ahmed\\Desktop\\DS\\Hackathon\\')
    vs = cv2.VideoCapture('yolov5/video/HD CCTV Traffic.mp4')
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # Load model
    model = torch.load('yolov5/last_yolov5s_results.pt', map_location=device)['model'].float()  # load to FP32
    model.to(device).eval()
    
    labels = pd.DataFrame(columns=['image','label'])
    path = 'static/cropped/'

    j = 0 
    while True:
        (grabbed,frame) = vs.read()
        if not grabbed:
            break
        img = cv2.resize(frame,(416,416))
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(device)
        img =  img.float()  # uint8 to fp16/32
        img /= 255.0   
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        pred = model(img, augment=False)[0]
        pred = non_max_suppression(pred, 0.5, 0.6)
        boxes = []
        scores = []
        lb = []
        for i, det in enumerate(pred):  # detections per image
        # save_path = 'draw/' + image_id + '.jpg'
            if det is not None and len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], frame.shape)

                # Write results
                for *xyxy, conf, cls in det:
                    boxes.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3])])
                    scores.append(conf)
                    lb.append(cls)
        boxes = np.array(boxes)
        scores = np.array(scores)
        lb = np.array(lb)
        boxes = boxes[scores >= 0.05].astype(np.int32)
        lb = lb[scores >=float(0.05)]
        scores = scores[scores >=float(0.05)]
        for i in range(len(boxes)):
            j += 1
            x = boxes[i][0]
            y = boxes[i][1]
            w = boxes[i][2]
            h = boxes[i][3]
            imgg = frame[y:y+h, x:x+w]
            try:
                labels.at[j,'image'] = 'img' + str(j)
                labels.at[j,'label'] = lb[i]
                fname = 'img'+str(j)+'.png'
                cv2.imwrite(path+fname,imgg)
            except:
                break

In [5]:
def run():
    os.chdir('C:\\Users\\Muhammad Ahmed\\Desktop\\DS\\Hackathon\\')
    model = load_learner('','vehicle_classifier.pkl')
    model1=load_learner('','emergency_classifier.pkl')
    count=-1
    import cv2,random, tqdm
    results = pd.DataFrame(columns = ['S No','Name','License_No','Vehicle Type','Emergency','License Plate'])
    for files in os.listdir('static/cropped'):
        sub_path=os.path.join('static/cropped/','')
    
        for sub_files in tqdm.tqdm(os.listdir(sub_path)):
            count+=1
            img=open_image(os.path.join(sub_path,sub_files))
            dataset=['Ambulance', 'Bus', 'Car', 'Limousine', 'Motorcycle', 'Taxi', 'Truck', 'Van']
            emerge=['Non-Emergency','Emergency']
            tens=model.predict(img)[-1].numpy()
            tens1=model1.predict(img)[-1].numpy()
            with open('test_db.csv','a') as f:
                try:
                    gsfi = get_string_from_image(os.path.join(sub_path,sub_files))
                    if len(gsfi)>2:
                        no=random.randint(1,50000)
                        lic="IND_{}".format(no)
                        #sql_insert(count,l[no][:-2],dataset[np.argmax(tens)],lic,emerge[np.argmax(tens1)],get_string_from_image(os.path.join(sub_path,sub_files)))
                        results.at[count,:] = [count,l[no][:-2],dataset[np.argmax(tens)],lic,emerge[np.argmax(tens1)],get_string_from_image(os.path.join(sub_path,sub_files))]             
                except:
                    continue

In [6]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [7]:
def create_project(conn, project):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    try:
        sql = ''' INSERT INTO Data(Name,Email,Password,Adhaar_Card_Number)
                  VALUES(?,?,?,?) '''
        cur = conn.cursor()
        cur.execute(sql, project)
        conn.commit()
    except Error as e:
        print(e)
    return cur.lastrowid

In [8]:
def create_project_2(conn, project, password_entered):
    cur = conn.cursor()
    cur.execute("SELECT Password FROM Data WHERE Email=?", (project,))
    password = cur.fetchall()
    if password[0][0] == password_entered:
        return True
    return False

In [9]:
def create_project_3(conn, project):
    cur = conn.cursor()
    cur.execute("SELECT Fine FROM Fines WHERE Email=?", (project,))
    payment = cur.fetchall()
    return payment

In [10]:
server = Flask(__name__)

In [11]:
@server.route('/')
def home():
    return render_template('main.html')

@server.route('/index.html', methods=['GET', 'POST'])
def admin():
    return render_template('index.html')

@server.route('/blank.html', methods=['GET', 'POST'])
def blank():
    return render_template('blank.html')

@server.route('/login', methods=['GET','POST'])
def login():
    email = request.form['email']
    password = request.form['Password']
    
    conn = create_connection('UsersData.db')
    task_1 = email
    
    res = create_project_2(conn, task_1, password)
    if res == True:
        if email == 'admin@admin.com':
            return render_template('index.html')
        payment = create_project_3(conn, email)[0][0]
        return render_template('payment.html',data=payment)
    return render_template('main.html')

@server.route('/register', methods=['GET','POST'])
def registerDone():
    os.chdir('C:\\Users\\Muhammad Ahmed\\Desktop\\DS\\Hackathon\\')
    name = request.form['name']
    email = request.form['email']
    password = request.form['password']
    number = request.form['Number']
    
    conn = create_connection('UsersData.db')
    task_1 = (name, email, password, number)
    
    create_project(conn, task_1)
    
    return render_template('main.html')

@server.route('/register.html', methods=['GET', 'POST'])
def register():
    return render_template('register.html')

@server.route('/extract_vehicles')
def extract_func():
    detect()
    return render_template('blank.html')

@server.route('/extract_vehicles_1')
def extract_vehicles():
    extract()
    message = request.args.get("Vehicles Extracted")
    return render_template('blank.html', msg=message)

@server.route('/run_operations')
def run_operations():
    return render_template('success.html', msg=message)

In [13]:
@server.route("/home.html")
def chatbot():
    return render_template("home.html")

@server.route("/get")
def get_bot_response(msg):
    return str(chat(msg))

In [15]:
if __name__ == "__main__":
    server.static_folder = 'static'
    server.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Sep/2020 23:53:42] "GET /home.html HTTP/1.1" 200 -
